In [ ]:
import tensorflow as tf
import keras_nlp
import numpy as np

In [ ]:
input = np.ones((16, 4, 150))
input[:, 3:, :] = 0
input[:, 1:, 100:] = 0
input[12:] = 0 
input = tf.constant(input)
sequence_masks = tf.cast(tf.not_equal(input, 0), tf.float32)
indicies = tf.multiply(tf.ones_like(sequence_masks), tf.range(0, 150, dtype=tf.float32))
indicies = tf.multiply(indicies, sequence_masks)
indicies = tf.argmax(indicies, axis=2)
indicies
emb = tf.keras.layers.Embedding(5, 128, input_length=150, mask_zero=False)
pos_embedding = keras_nlp.layers.PositionEmbedding(sequence_length=150)

output = emb(input)
pos_emb = pos_embedding(output)
output = tf.add(output, pos_emb)
output

In [ ]:
lstm = tf.keras.layers.LSTM(128, return_sequences=True)

@tf.function()
def map_lstm(inputs):
    input, indicies = inputs
    tf.print(indicies)
    output = lstm(input)
    return (tf.gather(output, indices=indicies, batch_dims=1), indicies)

map_input = tf.stack([output, indicies])
tf.map_fn(map_lstm, map_input, fn_output_signature=(tf.TensorSpec(dtype=tf.float32, shape=[None, 128]),
    tf.TensorSpec(dtype=tf.int64, shape=[None])))


In [ ]:
fn=lambda t: map_lstm(t)
@tf.function
def func(elems):
  return tf.map_fn(fn, elems, parallel_iterations=3, fn_output_signature=tf.TensorSpec(dtype=tf.float32, shape=[None, 128]))
func((output, indicies))
